# Kind Cluster erstellen

In [ ]:
. kubernetes-utils
create-kind-cluster traefik2-test

export KUBECONFIG="$(kind get kubeconfig-path --name="traefik2-test")"
kubectl cluster-info

In [ ]:
kubectl create ns traefik
kubectl apply -f crd.yaml -n traefik
kubectl apply -f service.yaml -n traefik
kubectl apply -f deployment.yaml -n traefik
kubectl rollout status deploy traefik -n traefik

In [ ]:
helm repo add maesh https://containous.github.io/maesh/charts
helm repo update
helm tiller start-ci
helm install --name=maesh --namespace=maesh maesh/maesh
helm tiller stop

In [ ]:
helm fetch \
  --repo https://containous.github.io/maesh/charts \
  --untar \
  --untardir ./charts \
    maesh
if [ ! -d manifests ]; then mkdir manifests; fi
helm template \
  --values ./charts/maesh/values.yaml \
  --output-dir ./manifests \
    ./charts/maesh

In [ ]:
kubectl apply --recursive --filename ./manifests/maesh -n maesh
kubectl rollout status deploy maesh-controller -n maesh

In [ ]:
kubectl annotate service \
    whoami maesh.containo.us/traffic-type=http -n traefik

In [ ]:
kubectl apply -f ingress-route.yaml -n traefik

In [ ]:
kubectl patch service prometheus -n maesh --patch "$(cat prometheus-service.yaml )"
kubectl apply -f prometheus.yaml -n maesh
kubectl rollout status deploy prometheus-core -n maesh

In [ ]:
kubectl patch service grafana -n maesh --patch "$(cat grafana-service.yaml )"
kubectl apply -f grafana.yaml -n maesh
kubectl rollout status deploy grafana-core -n maesh

In [ ]:
delete-kind-cluster